To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Unsloth now supports [gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which automatically creates kernels!

[Vision RL](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) is now supported! Train Qwen2.5-VL, Gemma 3 etc. with GSPO or GRPO.

Introducing Unsloth [Standby for RL](https://docs.unsloth.ai/basics/memory-efficient-rl): GRPO is now faster, uses 30% less memory with 2x longer context.

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
# # !cd /lambda/nfs/DiskUsEast1/finetuning_evaluation
# !python3 -m venv venv_3_Pure_Grit
# !source venv_3_Pure_Grit/bin/activate
# # !cd /lambda/nfs/DiskUsEast1/finetuning_evaluation/

In [2]:
# Pure GRIT - Install from official GRIT requirements.txt
import os

if "COLAB_" not in "".join(os.environ.keys()):
    # A10 GPU environment - install GRIT dependencies
    !pip install -r requirements.txt  # ✅ Use GRIT's official requirements
    !pip install tensorboard tensorboardX   # Add TensorBoard for logging
    !pip install python-dotenv
    print("✅ Pure GRIT dependencies installed from requirements.txt")
else:
    # Colab fallback
    !pip install -r requirements.txt
    !pip install tensorboard tensorboardX
    !pip install python-dotenv

# Verify installations
!pip check

✅ Pure GRIT dependencies installed from requirements.txt
No broken requirements found.


In [3]:
# ✅ ADD: GRIT integration - add to Python path
import sys
sys.path.insert(0, './grit')

### Unsloth

In [4]:
# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
#     "unsloth/llama-3-8b-Instruct-bnb-4bit",
#     "unsloth/llama-3-70b-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
# ] # More models at https://huggingface.co/unsloth

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/llama-3-8b-bnb-4bit",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

In [5]:
# Pure GRIT - Use standard HuggingFace loading (no Unsloth)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv('/lambda/nfs/DiskUsEast1/finetuning_evaluation/.env')
hf_token = os.getenv('HF_TOKEN')

max_seq_length = 2048
dtype = torch.bfloat16  # BFloat16 for A10 GPU (Ampere architecture)
load_in_4bit = True

# BitsAndBytes 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load base Llama-3 8B model
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",  # ✅ Official Meta model
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=dtype,
    token=hf_token,  # ✅ Use token from .env
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    use_fast=True,
    token=hf_token,  # ✅ Use token for tokenizer too
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"  # For batch generation

print(f"✅ Model loaded: {model.__class__.__name__}")
print(f"✅ Device map: {model.hf_device_map}")
print(f"✅ Tokenizer vocab size: {len(tokenizer)}")

/home/ubuntu/DiskUsEast1/finetuning_evaluation/venv_3_Pure_Grit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.52s/it]


✅ Model loaded: LlamaForCausalLM
✅ Device map: {'': 0}
✅ Tokenizer vocab size: 128256


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [6]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 16,
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
# )

In [7]:
# Clear GPU cache in your current notebook

# Add this BEFORE the LoRA configuration cell:

# Clear GPU memory before LoRA setup
import gc
import torch

gc.collect()
torch.cuda.empty_cache()
print(f"GPU memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
print(f"GPU memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")

GPU memory allocated: 5.31 GB
GPU memory reserved: 7.10 GB


#### Pure GRIT LoRA configuration

In [8]:
# Pure GRIT - Use PEFT's native LoRA (no Unsloth wrapper)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
import gc

# Clear cache before preparation
gc.collect()
torch.cuda.empty_cache()

# Prepare model for k-bit training
print("Preparing model for k-bit training...")
model = prepare_model_for_kbit_training(
    model,
    use_gradient_checkpointing=True,  # Enable gradient checkpointing
)

# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,  # Match baseline (Unsloth used 16, train_alpaca.py uses 32)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

# Apply LoRA adapters
print("Applying LoRA adapters...")
model = get_peft_model(model, lora_config)
model.enable_input_require_grads()
model.config.use_cache = False  # Required for gradient checkpointing

# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()

# Print trainable parameters
model.print_trainable_parameters()

# Final memory check
gc.collect()
torch.cuda.empty_cache()
print(f"✅ LoRA adapters added successfully")
print(f"GPU memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
print(f"GPU memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")

Preparing model for k-bit training...
Applying LoRA adapters...
trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196
✅ LoRA adapters added successfully
GPU memory allocated: 7.43 GB
GPU memory reserved: 11.17 GB


#### GRIT Manager initialization

In [9]:
# ✅ Cell 11 - Initialize GRIT Manager
from grit.config import GRITConfig
from grit.manager import GRITManager

# Initialize config
grit_config = GRITConfig()

# Model settings - match LoRA config
grit_config.lora_rank = 16
grit_config.lora_alpha = 16
grit_config.precision = "bf16"

# CONSERVATIVE SETTINGS for 200-step run
grit_config.kfac_update_freq = 10         # Moderate frequency
grit_config.reprojection_freq = 50        # Less frequent than hybrid
grit_config.kfac_damping = 1e-5           # Low damping for visible effect
grit_config.lambda_kfac = 1e-6            # Minimal K-FAC regularization
grit_config.lambda_reproj = 1e-5          # Minimal reprojection regularization
grit_config.kfac_min_samples = 16         # Lower than default (64), higher than aggressive (4)

# Warmup settings
grit_config.reprojection_warmup_steps = 20  # Skip early reprojection
grit_config.ng_warmup_steps = 0             # No warmup for natural gradient
grit_config.regularizer_warmup_steps = 0    # No warmup for regularizers

# Rank adaptation - DISABLED for fair comparison
grit_config.enable_rank_adaptation = False
grit_config.use_two_sided_reprojection = True

# Logging - DISABLED to reduce overhead
grit_config.log_fisher_spectrum = False
grit_config.log_top_eigs = 0
grit_config.log_eig_heatmaps = False

print("🎯 Initializing GRIT Manager...")
grit_manager = GRITManager(
    model=model,
    config=grit_config,
    device="cuda",
)
print("✅ GRIT Manager initialized successfully!")

🎯 Initializing GRIT Manager...
Instrumented 224 LoRA modules with custom autograd.
Using r-dim (16x16) covariances.
GRITManager: Initialization complete.
🔍 Optimizing 224 key LoRA modules.
💾 K-FAC covariances kept on-device; snapshot to CPU only at inversion.
✅ GRIT Manager initialized successfully!


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [vicgalle](https://huggingface.co/datasets/vicgalle/alpaca-gpt4), which is a version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html) generated from GPT4. You can replace this code section with your own data prep.

In [10]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("vicgalle/alpaca-gpt4", split="train")
print(f"📊 Loaded dataset: {len(dataset)} samples")

# Tokenization function
def tokenize_function(example):
    instr = example['instruction'].strip()
    inp = example.get('input', "").strip()
    out = example['output'].strip()

    # Create prompt in Alpaca format
    if inp:
        prompt = f"### Instruction:\n{instr}\n\n### Input:\n{inp}\n\n### Response:\n{out}"
    else:
        prompt = f"### Instruction:\n{instr}\n\n### Response:\n{out}"

    # Tokenize with padding/truncation
    tokenized = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=max_seq_length,
        return_tensors=None,
    )

    # Labels = input_ids for causal LM training
    tokenized["labels"] = tokenized["input_ids"][:]

    return tokenized

# Apply tokenization to entire dataset
print("🔄 Tokenizing dataset...")
tokenized_dataset = dataset.map(
    tokenize_function,
    remove_columns=dataset.column_names,
    desc="Tokenizing dataset",
    batched=False,
)

print(f"✅ Dataset tokenized: {len(tokenized_dataset)} samples")
print(f"✅ Sample keys: {tokenized_dataset.column_names}")
print(f"✅ Input shape: {len(tokenized_dataset[0]['input_ids'])}")

📊 Loaded dataset: 52002 samples
🔄 Tokenizing dataset...


Tokenizing dataset: 100%|██████████| 52002/52002 [01:13<00:00, 703.10 examples/s] 

✅ Dataset tokenized: 52002 samples
✅ Sample keys: ['input_ids', 'attention_mask', 'labels']
✅ Input shape: 2048


In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("vicgalle/alpaca-gpt4", split="train")
# print(dataset.column_names)

['instruction', 'input', 'output', 'text']


One issue is this dataset has multiple columns. For `Ollama` and `llama.cpp` to function like a custom `ChatGPT` Chatbot, we must only have 2 columns - an `instruction` and an `output` column.

In [ ]:
# print(dataset.column_names)

['instruction', 'input', 'output', 'text']


To solve this, we shall do the following:
* Merge all columns into 1 instruction prompt.
* Remember LLMs are text predictors, so we can customize the instruction to anything we like!
* Use the `to_sharegpt` function to do this column merging process!

For example below in our [Titanic CSV finetuning notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb), we merged multiple columns in 1 prompt:

<img src="https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Merge.png" height="100">

To merge multiple columns into 1, use `merged_prompt`.
* Enclose all columns in curly braces `{}`.
* Optional text must be enclused in `[[]]`. For example if the column "Pclass" is empty, the merging function will not show the text and skp this. This is useful for datasets with missing values.
* You can select every column, or a few!
* Select the output or target / prediction column in `output_column_name`. For the Alpaca dataset, this will be `output`.

To make the finetune handle multiple turns (like in ChatGPT), we have to create a "fake" dataset with multiple turns - we use `conversation_extension` to randomnly select some conversations from the dataset, and pack them together into 1 conversation.

In [ ]:
# from unsloth import to_sharegpt

# dataset = to_sharegpt(
#     dataset,
#     merged_prompt="{instruction}[[\nYour input is:\n{input}]]",
#     output_column_name="output",
#     conversation_extension=3,  # Select more to handle longer conversations
# )

Finally use `standardize_sharegpt` to fix up the dataset!

In [ ]:
# from unsloth import standardize_sharegpt

# dataset = standardize_sharegpt(dataset)

### Customizable Chat Templates

You also need to specify a chat template. Previously, you could use the Alpaca format as shown below.

In [ ]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

Now, you have to use `{INPUT}` for the instruction and `{OUTPUT}` for the response.

We also allow you to use an optional `{SYSTEM}` field. This is useful for Ollama when you want to use a custom system prompt (also like in ChatGPT).

You can also not put a `{SYSTEM}` field, and just put plain text.

```python
chat_template = """{SYSTEM}
USER: {INPUT}
ASSISTANT: {OUTPUT}"""
```

Use below if you want to use the Llama-3 prompt format. You must use the `instruct` and not the `base` model if you use this!
```python
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""
```

For the ChatML format:
```python
chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""
```

The issue is the Alpaca format has 3 fields, whilst OpenAI style chatbots must only use 2 fields (instruction and response). That's why we used the `to_sharegpt` function to merge these columns into 1.

In [ ]:
# chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

# ### Instruction:
# {INPUT}

# ### Response:
# {OUTPUT}"""

# from unsloth import apply_chat_template

# dataset = apply_chat_template(
#     dataset,
#     tokenizer=tokenizer,
#     chat_template=chat_template,
#     # default_system_message = "You are a helpful assistant", << [OPTIONAL]
# )

Unsloth: We automatically added an EOS token to stop endless generations.


##### ✅ TensorBoard Setup - Centralized logging for all 3 models

In [ ]:
import os
from datetime import datetime

# Create centralized tensorboard directory at home
tensorboard_base_dir = "/home/ubuntu/DiskUsEast1/finetuning_evaluation/tensorboard_logs"
os.makedirs(tensorboard_base_dir, exist_ok=True)

# Create run-specific directory with timestamp
run_name = "QLoRA_GRIT"  # Change for each model: "unsloth_ft", "hybrid_grit", "pure_grit"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
tensorboard_run_dir = os.path.join(tensorboard_base_dir, f"{run_name}_{timestamp}")

print(f"📊 TensorBoard logs will be saved to: {tensorboard_run_dir}")

📊 TensorBoard logs will be saved to: /home/ubuntu/DiskUsEast1/finetuning_evaluation/tensorboard_logs/pure_grit_20251002_072335


<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
# from transformers import TrainerCallback
            
# class GritVerificationCallback(TrainerCallback):
#     """Monitor GRIT preconditioning during training with detailed verification"""

#     def __init__(self, grit_manager):
#         self.grit_manager = grit_manager
#         self.preconditioning_count = 0
#         self.inversion_count = 0

#     def on_step_end(self, args, state, control, **kwargs):
#         # Update Fisher matrices
#         last_loss = state.log_history[-1].get("loss") if state.log_history else None
#         old_last = self.grit_manager.last_kfac_update_step
#         self.grit_manager.step(loss=last_loss)

#         # Check if Fisher inversion happened
#         if self.grit_manager.last_kfac_update_step > old_last:
#             self.inversion_count += 1
#             print(f"\n[Step {state.global_step}] ✓ Fisher inversion #{self.inversion_count}")

#     def on_optimizer_step(self, args, state, control, **kwargs):
#         # Apply natural gradient preconditioning
#         if self.grit_manager.factors_are_ready:
#             self.grit_manager.precondition_gradients()
#             self.preconditioning_count += 1
#             if state.global_step % 10 == 0:
#                 print(f"[Step {state.global_step}] ✓ Preconditioning applied (#{self.preconditioning_count})")
#         else:
#             if state.global_step <= 20 or state.global_step % 10 == 0:
#                 print(f"[Step {state.global_step}] ✗ Preconditioning skipped (not ready)")

#     def on_train_end(self, args, state, control, **kwargs):
#         print(f"\n{'='*80}")
#         print(f"VERIFICATION SUMMARY:")
#         print(f"  Total steps:        {state.global_step}")
#         print(f"  Fisher inversions:  {self.inversion_count}")
#         print(f"  Preconditioning:    {self.preconditioning_count}")
#         print(f"  Status:             {'✓ WORKING' if self.preconditioning_count > 0 else '✗ NOT WORKING'}")
#         print(f"{'='*80}\n")

#### Trainer Setup

In [ ]:
# from trl import SFTConfig, SFTTrainer
# # ✅ ADD: Import GRIT callback
# from grit.trainer import GritCallback

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = SFTConfig(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         max_steps = 200, # ✅  decrease from 200 to 5 for TESTing purpose
#         # num_train_epochs = 1, # For longer training runs!
#         learning_rate = 2e-4,
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
        
#         # ✅ TensorBoard configuration
#         report_to = "tensorboard",  # Changed from None to "tensorboard"
#         logging_dir = tensorboard_run_dir,  # Point to centralized location
#         logging_first_step = True,
#     ),
#     # ✅ ADD: Inject GRIT callback (non-invasive!)
#     callbacks = [GritVerificationCallback(grit_manager)],
# )

In [13]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from grit.trainer import GritTrainer
import torch

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    return_tensors="pt",
)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./outputs/pure_grit",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=200,
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    logging_steps=1,
    optim="adamw_torch",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    save_strategy="steps",
    save_steps=100,
    logging_dir=tensorboard_run_dir,  # Use auto-generated timestamp
    report_to="tensorboard",
)

# Initialize GritTrainer
trainer = GritTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    grit_manager=grit_manager,
)

print("✅ GritTrainer initialized")
print(f"📊 Training steps: {training_args.max_steps}")
print(f"📊 Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")

/lambda/nfs/DiskUsEast1/finetuning_evaluation/unsloth_3_Pure_Grit/grit/trainer.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `GritTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


GritTrainer: Initialized with GRIT implementation.
✅ GritTrainer initialized
📊 Training steps: 200
📊 Effective batch size: 8


In [15]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}")
print(f"Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A10
Max memory = 22.069 GB.
11.166 GB of memory reserved.


In [16]:
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.


🎁 Wrapping the optimizer with GRIT preconditioning logic.


Step,Training Loss
1,7.495700
2,7.142900
3,5.444800
4,2.399800
5,0.769700
6,0.189600
7,0.127400
8,0.147300
9,0.143200
10,0.131800


##### View TensorBoard

In [17]:
# ✅ Launch TensorBoard to compare all models
print("🎯 To view TensorBoard:")
print(f"Run in terminal: tensorboard --logdir={tensorboard_base_dir} --port=6006")
print(f"Then open: http://localhost:6006")
print("")
print("📊 All training runs:")
!ls -lh {tensorboard_base_dir}

🎯 To view TensorBoard:
Run in terminal: tensorboard --logdir=/home/ubuntu/DiskUsEast1/finetuning_evaluation/tensorboard_logs --port=6006
Then open: http://localhost:6006

📊 All training runs:
total 1.5K
drwxrwxr-x 2 ubuntu ubuntu 1 Oct  2 04:20 hybrid_grit_20251002_042053
drwxrwxr-x 2 ubuntu ubuntu 1 Oct  2 07:27 pure_grit_20251002_072335
drwxrwxr-x 2 ubuntu ubuntu 1 Oct  2 04:40 unsloth_ft_20251002_044038


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

978.6059 seconds used for training.
16.31 minutes used for training.
Peak reserved memory = 7.373 GB.
Peak reserved memory for training = 0.627 GB.
Peak reserved memory % of max memory = 33.409 %.
Peak reserved memory for training % of max memory = 2.841 %.


<a name="Inference"></a>
### Inference
Let's run the model! Unsloth makes inference natively 2x faster as well! You should use prompts which are similar to the ones you had finetuned on, otherwise you might get bad results!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8,"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

Since we created an actual chatbot, you can also do longer conversations by manually adding alternating conversations between the user and assistant!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8"},
    {"role": "assistant", "content": "The fibonacci sequence continues as 13, 21, 34, 55 and 89."},
    {"role": "user",      "content": "What is France's tallest tower called?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [                    # Change below!
    {"role": "user", "content": "Describe anything special about a sequence. Your input is 1, 1, 2, 3, 5, 8,"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

<a name="Ollama"></a>
### Ollama Support

[Unsloth](https://github.com/unslothai/unsloth) now allows you to automatically finetune and create a [Modelfile](https://github.com/ollama/ollama/blob/main/docs/modelfile.md), and export to [Ollama](https://ollama.com/)! This makes finetuning much easier and provides a seamless workflow from `Unsloth` to `Ollama`!

Let's first install `Ollama`!

In [ ]:
# Install CURL library dependency before Ollama installation
!apt-get update && apt-get install -y libcurl4-openssl-dev

# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

Next, we shall save the model to GGUF / llama.cpp

We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

We also support saving to multiple GGUF options in a list fashion! This can speed things up by 10 minutes or more if you want multiple export formats!

In [ ]:
# Build llama.cpp before saving to GGUF
# Clean and rebuild llama.cpp
!cd llama.cpp && rm -rf build && mkdir build && cd build && cmake .. -DLLAMA_CURL=OFF && cmake --build . --config Release -j$(nproc)
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

We use `subprocess` to start `Ollama` up in a non blocking fashion! In your own desktop, you can simply open up a new `terminal` and type `ollama serve`, but in Colab, we have to use this hack!

In [ ]:
import subprocess

subprocess.Popen(["ollama", "serve"])
import time

time.sleep(3)  # Wait for a few seconds for Ollama to load!

`Ollama` needs a `Modelfile`, which specifies the model's prompt format. Let's print Unsloth's auto generated one:

In [ ]:
print(tokenizer._ollama_modelfile)

We now will create an `Ollama` model called `unsloth_model` using the `Modelfile` which we auto generated!

In [ ]:
!ollama create unsloth_model -f ./model/Modelfile

And now we can do inference on it via `Ollama`!

You can also upload to `Ollama` and try the `Ollama` Desktop app by heading to https://www.ollama.com/

In [ ]:
!curl http://localhost:11434/api/chat -d '{ \
    "model": "unsloth_model", \
    "messages": [ \
        { "role": "user", "content": "Continue the Fibonacci sequence: 1, 1, 2, 3, 5, 8," } \
    ] \
    }'

# ChatGPT interactive mode

### ⭐ To run the finetuned model like in a ChatGPT style interface, first click the **| >_ |** button.
![](https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Where_Terminal.png)

---
---
---

### ⭐ Then, type `ollama run unsloth_model`

![](https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Terminal_Type.png)

---
---
---
### ⭐ And you have a ChatGPT style assistant!

### Type any question you like and press `ENTER`. If you want to exit, hit `CTRL + D`
![](https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/Assistant.png)You can also use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
